In [1]:
%matplotlib inline
import os
import numpy as np
from matplotlib import pyplot as plt
#
wd = os.getcwd()     # save current working directory
os.chdir('../src')   # change to directory containing our tools 
import simutils
import emio
os.chdir(wd)         # change back to working directory

ModuleNotFoundError: No module named 'mdtraj'

In [ ]:
simulator_bin='~/gdrive_perso/Toolkit/Microscope_Simulator/TEM-simulator_1.3/src/TEM-simulator'

# Simulation setup

## Setting parameters

### molecular model

In [16]:
pdbdir     = 'material/simple_data/' # relative path to directory containing candidate PDB fiels
pdb        = '4v6x'                  # ID of PDB file found in pdbdir
voxel_size      = 0.1                # [nm]
particle_name   = 'toto'             # Name of the particle. Not very important.
particle_mrcout = None               # volume map of sample is written.

### specimen grid parameters

In [17]:
hole_diameter         = 1200 # [nm]
hole_thickness_center = 100  # [nm]
hole_thickness_edge   = 100  # [nm]

### beam parameters

In [18]:
voltage           = 300 # [kV]
energy_spread     = 1.3 # [V]
electron_dose     = 100 # [e/nm**2] dose per image
electron_dose_std = 0   # standard deviation of dose per image

### optics parameters

In [19]:
magnification         = 81000 #
spherical_aberration  = 2.7   # [mm]
chromatic_aberration  = 2.7   # [mm]
aperture_diameter     = 50    # [um] in back focal plane
focal_length          = 3.5   # [mm]
aperture_angle        = 0.1   # [mrad] of the condenser lens
defocus               = 1.0   # [um]
defocus_syst_error    = 0.0   #
defocus_nonsyst_error = 0.0   #
optics_defocusout     = None  # file to write defocus values

### detector parameters

In [20]:
detector_Nx           = 5760 # number of pixels along X axis
detector_Ny           = 4092 # number of pixels along Y axis
detector_pixel_size   = 5    # [um]
detector_gain         = 32   # average number of counts per electron
detector_Q_efficiency = 0.5  # detector quantum efficiency
noise                 = 'no' # whether quantized electron waves result in noise
MTF_params            = [0,0,1,0,0] # to be described. [0,0,1,0,0] is perfect detector.

### output

In [21]:
mrcdir = '/Users/fpoitevi/gdrive/cryoEM/Projects/Simu-rib80s/synthetic_datasets/dataset1/'

## Assemble a few input arrays

In [23]:
sample_dimensions = [ hole_diameter, hole_thickness_center, hole_thickness_edge ]
beam_params       = [ voltage, energy_spread, electron_dose, electron_dose_std ]
optics_params     = [ magnification, spherical_aberration, chromatic_aberration, 
                      aperture_diameter, focal_length, aperture_angle,
                      defocus, defocus_syst_error, defocus_nonsyst_error, optics_defocusout ]
detector_params   = [ detector_Nx, detector_Ny, detector_pixel_size, 
                      detector_gain, noise, detector_Q_efficiency,
                      MTF_params[0], MTF_params[1], MTF_params[2], MTF_params[3], MTF_params[4] ]
xy_range, numpart = simutils.define_grid(sample_dimensions,
                                         pdb_file=pdb_file,
                                         Dmax=30)

NameError: name 'simutils' is not defined

# Simulate data

## Prepare input file

### 1. Define the filenames

In [ ]:
pdb_file, mrc_file, crd_file, log_file, inp_file = emio.simio(pdbdir, 
                                                              pdb, 
                                                              pdb+'_copy'+str(copy),
                                                              mrcdir, 
                                                              pdb+'_copy'+str(copy)+'_defoc'+str(defoc)+'_nowater')
if particle_mrcout is not None:
    particle_mrcout = mrc_file
#

### 2. Build specimen coordinate file (grid of particles)

In [ ]:
simutils.write_crd_file(numpart, xrange = xy_range, yrange = xy_range, crd_file = crd_file)

### 3. Build input dictionary

In [ ]:
params_dictionary = simutils.fill_parameters_dictionary(mrc_file = mrc_file, 
                                                        pdb_file = pdb_file, 
                                                        particle_mrcout = particle_mrcout, 
                                                        crd_file = crd_file,
                                                        sample_dimensions = sample_dimensions,
                                                        beam_params = beam_params,
                                                        optics_params = optics_params,
                                                        detector_params = detector_params,
                                                        log_file = log_file
                                                        seed=-1234)
#

### 4. Format dictionary in input file

In [ ]:
simutils.write_inp_file(inp_file = inp_file, dict_params = params_dictionary)

## Run the simulation

In [ ]:
os.system('{0} {1}'.format(simulator_bin, inp_file))